In [1]:
import configparser
from datetime import datetime
import os
import glob
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from datetime import datetime

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1587535371117_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'AWS'
Traceback (most recent call last):
  File "/usr/lib64/python3.6/configparser.py", line 959, in __getitem__
    raise KeyError(key)
KeyError: 'AWS'



In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
output_data_path = 's3a://myawstwstbucket/'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Load all parquet files created earlier

In [6]:
# users
parquetFile = spark.read.parquet(output_data_path+"users")
parquetFile.createOrReplaceTempView("users")
parquetFile.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)

In [7]:
# songs
parquetFile = spark.read.parquet(output_data_path+"songs")
parquetFile.createOrReplaceTempView("songs")
parquetFile.printSchema()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- artist_id: string (nullable = true)

In [8]:
# artists
parquetFile = spark.read.parquet(output_data_path+"artists")
parquetFile.createOrReplaceTempView("artists")
parquetFile.printSchema()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- lattitude: double (nullable = true)
 |-- longitude: double (nullable = true)

In [9]:
# time 
parquetFile = spark.read.parquet(output_data_path+"times")
parquetFile.createOrReplaceTempView("times")
parquetFile.printSchema()



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- start_time: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- weekday: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)

In [10]:
# songplays
parquetFile = spark.read.parquet(output_data_path+"songplays")
parquetFile.createOrReplaceTempView("songplays")
parquetFile.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- songplay_id: long (nullable = true)
 |-- start_time: timestamp (nullable = true)
 |-- user_id: string (nullable = true)
 |-- level: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- session_id: long (nullable = true)
 |-- location: string (nullable = true)
 |-- user_agent: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)

# Query

## Show some records

In [19]:
spark.sql("SELECT * FROM users").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+---------+------+-----+
|user_id|first_name|last_name|gender|level|
+-------+----------+---------+------+-----+
|     88|  Mohammad|Rodriguez|     M| free|
|     75|    Joseph|Gutierrez|     M| free|
|     69|  Anabelle|  Simpson|     F| free|
|     29|Jacqueline|    Lynch|     F| free|
|     68|    Jordan|Rodriguez|     F| free|
+-------+----------+---------+------+-----+
only showing top 5 rows

In [20]:
spark.sql("SELECT * FROM songs WHERE year = 1969").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+---------+----+------------------+
|           song_id|               title| duration|year|         artist_id|
+------------------+--------------------+---------+----+------------------+
|SOCGTUJ12A6D4F7C61|In The Evening (I...|273.03138|1969|ARWHYTK1187FB482B3|
|SOOZFGU12A6D4F9CCF|Stop This Train (...|365.84444|1969|ARSMEBG1187FB4D02B|
|SORNVVT12AF72A0C39|Intro 2/ Kick Out...|172.30322|1969|ARB336A1187B9A4583|
|SOWYSGV12A6701D47C|I Don't Know Wher...|193.41016|1969|ARQXK0B1187B9ACC97|
|SONJQSB12AF72A1F13|Rockin' Chair (20...|223.34649|1969|ARGF9VF1187FB37DAE|
|SOFULFU12A58A77951|Candy Man (Live A...|155.53261|1969|AR7HM181187FB3CFC1|
|SOXOCMB12AB017B90C|It Don't Take Muc...|160.57424|1969|ARVNNXD1187B9AE50D|
|SOQADJX12A6D4FE23A|The Driving Of Th...|114.38975|1969|ARA79JA1187FB38652|
|SOKGPSO12AB0187077|Garden of Earthly...|163.94404|1969|AROFSLD1187B99018B|
|SORRPFB12AB018341B|Blues for Passing...|381.46567|1969|ARX0VKO1187B9897CF|
|SOOANYZ12A6

In [21]:
spark.sql("SELECT * FROM artists").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+--------------------+---------+----------+
|         artist_id|                name|            location|lattitude| longitude|
+------------------+--------------------+--------------------+---------+----------+
|ARVXKTV1187FB44139|Arsonists Get All...|      Santa Cruz, CA| 36.97402|-122.03095|
|ARYLTG61187FB44145|       Brenda Fassie|Langa, Cape Town,...|-33.93333|  18.51667|
|ARUS5MJ1187B9B0A5D|Martina Topley Bi...|Knowle West, Bris...| 51.43558|  -2.57518|
|ARR76SY1187B9AD510|Kings Of Convenie...|      Bergen, Norway|  60.3907|   5.33275|
|ARHRNT11187FB5BEE7|  Daphne Loves Derby|    KENT, WASHINGTON| 47.38028|-122.23742|
+------------------+--------------------+--------------------+---------+----------+
only showing top 5 rows

In [22]:
spark.sql("SELECT * FROM times").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+---+----+-------+----+-----+
|          start_time|hour|day|week|weekday|year|month|
+--------------------+----+---+----+-------+----+-----+
|2018-11-15 11:15:...|  11| 15|  46|      5|2018|   11|
|2018-11-15 17:42:...|  17| 15|  46|      5|2018|   11|
|2018-11-15 19:13:...|  19| 15|  46|      5|2018|   11|
|2018-11-15 22:35:...|  22| 15|  46|      5|2018|   11|
|2018-11-21 01:48:...|   1| 21|  47|      4|2018|   11|
+--------------------+----+---+----+-------+----+-----+
only showing top 5 rows

In [23]:
spark.sql("SELECT * FROM songplays").show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+----+-----+
| songplay_id|          start_time|user_id|level|           song_id|         artist_id|session_id|            location|          user_agent|year|month|
+------------+--------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+----+-----+
|566935683072|2018-11-20 08:05:...|     15| paid|SOMCKUB12AB018BD24|ARL1MEI1187B9AD98B|       716|Chicago-Napervill...|"Mozilla/5.0 (X11...|2018|   11|
+------------+--------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+----+-----+
only showing top 1 row

In [11]:
## Number of female users in the data set?

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
spark.sql("SELECT COUNT(DISTINCT(user_id)) AS num_Female_users \
        FROM users \
        WHERE gender = 'F'").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+
|num_Female_users|
+----------------+
|              55|
+----------------+

In [13]:
## Number of songs were played from the most played artist?

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
spark.sql("SELECT a.name, COUNT(s.song_id) AS num_songs \
        FROM artists AS a \
        JOIN songs AS s \
        ON a.artist_id = s.artist_id \
        GROUP BY a.artist_id, a.name \
        ORDER BY num_songs DESC \
        ").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+
|                name|num_songs|
+--------------------+---------+
|Nick Cave & The B...|       16|
|            The Bats|       10|
|    Butthole Surfers|       10|
|     Badly Drawn Boy|        9|
|      Polygon Window|        9|
+--------------------+---------+
only showing top 5 rows

In [15]:
## Average number of hour users spent on listening song 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
spark.sql("SELECT AVG(hour) AS avg_hour_spent FROM \
    (SELECT u.user_id, t.hour \
        FROM songplays AS ss \
        JOIN times AS t \
        ON ss.start_time = t.start_time \
        JOIN users AS u \
        ON ss.user_id = u.user_id \
        GROUP BY u.user_id, t.hour )\
        ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+
|   avg_hour_spent|
+-----------------+
|13.14218009478673|
+-----------------+

In [17]:
## Song title that were played most

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
spark.sql("SELECT s.title, COUNT(s.song_id) AS num_play \
        FROM songs AS s \
        JOIN songplays AS ss \
        ON ss.song_id = s.song_id \
          GROUP BY ss.song_id, s.title \
        ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------+
|               title|num_play|
+--------------------+--------+
|   It Was A Good Day|       1|
|Rozando La Eternidad|       1|
|    Beautiful Flower|       1|
|       Driver's Seat|       1|
|Always Will (LP V...|       1|
|      Born In Africa|       1|
|       Solar Powered|       1|
|                Lady|       1|
|Revolution Deaths...|       1|
|         These Words|       1|
|Possibilities (Th...|       1|
|                 Why|       1|
|           Mr. Jones|       4|
|Emergency (Album ...|       2|
|Crawling (Album V...|       1|
|                Love|       1|
|From The Ritz To ...|       3|
|Spinning (Album V...|       1|
|Catch You Baby (S...|       9|
|Planet Caravan (R...|       1|
+--------------------+--------+
only showing top 20 rows